In [ ]:
!pip install transformers
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install datasets

In [1]:
import transformers
import torch

from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, AutoPeftModelForCausalLM, get_peft_model, PeftConfig
from peft import IA3Model, IA3Config
from datasets import load_dataset

In [3]:
config = IA3Config(peft_type = "IA3", task_type = "CAUSAL_LM", target_modules = ["query_key_value", "dense_4h_to_h"], feedforward_modules = ["dense_4h_to_h"])

tokenizer = AutoTokenizer.from_pretrained('EleutherAI/polyglot-ko-1.3b')
model = AutoModelForCausalLM.from_pretrained('EleutherAI/polyglot-ko-1.3b', device_map = 'auto', torch_dtype = torch.float16, rope_scaling = {"type": "dynamic", "factor": 2.0})
ia3_model = get_peft_model(model, config)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
model.eval()

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 2048)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXDynamicNTKScalingRotaryEmbedding()
          (query_key_value): Linear(
            in_features=2048, out_features=6144, bias=True
            (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 6144x1 (GPU 0)])
          )
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
      

In [5]:
ia3_model

PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 2048)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-23): 24 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXDynamicNTKScalingRotaryEmbedding()
              (query_key_value): Linear(
                in_features=2048, out_features=6144, bias=True
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.cuda.FloatTensor of size 6144x1 (GPU 0)])
              )
              (dense): Linear(in_features=2048, out_featu

In [6]:
data = load_dataset('beomi/KoAlpaca-v1.1a')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [7]:
data = data.remove_columns(['url'])
data = data.map(lambda x: {"text" : f"### 질문:{x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>"})
data = data.map(lambda x: tokenizer(x['text']), batched = True)

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [8]:
data

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21155
    })
})

In [9]:
tokenizer.pad_token = tokenizer.eos_token
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))

In [11]:
trainer = transformers.Trainer(
    model = ia3_model,
    train_dataset = data['train'],
    args = transformers.TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 1,
        max_steps = 50,
        learning_rate = 6e-2,
        save_total_limit = 2,
        optim="adamw_torch",
        save_steps = 50,
        logging_steps = 1,
        output_dir = "/content/drive/MyDrive/ColabNotebooks/output"
    ),
    data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm = False)
)

trainer.train()

Step,Training Loss
1,1.746100
2,1.332000
3,1.767600
4,1.080100
5,1.586900
6,1.329100
7,2.371100
8,2.011700
9,1.702100
10,1.471700


TrainOutput(global_step=50, training_loss=2.01115234375, metrics={'train_runtime': 13.5946, 'train_samples_per_second': 7.356, 'train_steps_per_second': 3.678, 'total_flos': 209503617662976.0, 'train_loss': 2.01115234375, 'epoch': 0.0})

In [2]:
peft_model_id = '/content/drive/MyDrive/ColabNotebooks/output/checkpoint-50'
config = PeftConfig.from_pretrained(peft_model_id)

In [6]:
config

IA3Config(peft_type='IA3', auto_mapping=None, base_model_name_or_path='EleutherAI/polyglot-ko-1.3b', revision=None, task_type='CAUSAL_LM', inference_mode=True, target_modules=['query_key_value', 'dense_4h_to_h'], feedforward_modules=['dense_4h_to_h'], fan_in_fan_out=False, modules_to_save=None, init_ia3_weights=True)

In [10]:
inf_model = AutoPeftModelForCausalLM.from_pretrained(peft_model_id, rope_scaling = {"type" : "dynamic", "factor" : 2.0}, device_map = 'auto')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

In [8]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    gened = inf_model.generate(
        **tokenizer(
            q,
            return_tensors = 'pt',
            return_token_type_ids = False
        ).to('cuda'),
        max_new_tokens = 256,
        early_stopping = True,
        do_sample = True,
        eos_token_id = 2
    )
    print(tokenizer.decode(gened[0]))

In [11]:
gen("몸에 단백질이 부족하면 나타나는 현상은?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 몸에 단백질이 부족하면 나타나는 현상은?

### 답변: 단백질 부족 때문일 수 있습니다. 하지만, 단백질의 형태는 '글로블럭'입니다. '글로블럭'은 단백질 외에도 콜레스테롤, 아말감, 글루칸 등 다양한 성분으로 구성되어 있습니다. 따라서 단백질만 채워지지 않았을 때 나타날 수 있는 문제점들을 점검해 보면 좋을 것 같습니다. 

### 답변: 단백질 부족은 혈중 글로블럭을 구성하는 필수 아미노산인 아미노산 중 하나인 메치오닌 등이 부족할 때 발생합니다. 이에 따라 근육이 감소거나 건강한 생활에 지장을 가져올 수 있는 것입니다. 따라서, 건강해지기 위한 꾸준한 단백질 섭취만이 건강에 도움이 될 것입니다.

 

출처: http://ko.haven.com/content-life-of-generalizing-generalization/출처: http://koreantm.hansol.cc/sphones/kaiser.htm.jpg### 답변: 육류와 유제품, 그리고 콩 등 다양한 단백질이 체표


In [ ]:
tokenizer.decode(ans[0]).replace("### 질문: 몸에 단백질이 부족하면 나타나는 현상은?\n\n### 답변", "")

': 단백질이 부족하면 나타나는 현상은 다음과 같습니다. 1.: 근육이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 근육량이 줄어들고, 근육이 줄어들면 체중이 줄어들고, 체중이 줄어들면 체중이 줄어들고, 체중이 줄어들면 체중이 줄어들고, 체중이 줄어들면 체중이 줄어들고, 체중'

## Inference